# Alt andet end lige: Data project

Importing packages and setting magics:

In [838]:
# importing the used packages 
import pandas as pd
import numpy as np
import datetime

# importing package to create plots and setting basic, visual settings
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"-"})
plt.rcParams.update({'font.size': 10})
import ipywidgets as widgets

# importing the API from DST used to import data
from dstapi import DstApi 

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

1. Data is imported using the API for Danmarks statistik

In [839]:
price = DstApi('EJ55') #EJ55 is data on the pricing of houses
unemp = DstApi('AULK01') #AULK01 is data on the number of unemployed 
unemp = DstApi('AUS08') #AUS08 is seasonally adjusted data on the unemployment rate

## Exploring the data sets

1. The availble values for each variable is plotted in order to select relevant variables. 

In [840]:
#An overview over the available data. 
tabsum = price.tablesummary(language='en')
display(tabsum)

# Displaying the available values for each variable:
for variable in tabsum['variable name']:
    print(variable+':')
    display(price.variable_levels(variable, language='en'))

Table EJ55: Price index for sales of property by region, category of real property, unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,17,000,All Denmark,11,Province Nordjylland,False
1,EJENDOMSKATE,3,0111,One-family houses,2103,"Owner-occupied flats, total",False
2,TAL,3,100,Index,310,Percentage change compared to same quarter the...,False
3,Tid,124,1992K1,1992Q1,2022K4,2022Q4,True


OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,02,Province Københavns omegn
4,03,Province Nordsjælland
5,04,Province Bornholm
6,085,Region Sjælland
7,05,Province Østsjælland
8,06,Province Vest- og Sydsjælland
9,083,Region Syddanmark


EJENDOMSKATE:


,id,text
0,0111,One-family houses
1,0801,Weekend cottages
2,2103,"Owner-occupied flats, total"


TAL:


,id,text
0,100,Index
1,210,Percentage change compared to previous quarter
2,310,Percentage change compared to same quarter the...


Tid:


,id,text
0,1992K1,1992Q1
1,1992K2,1992Q2
2,1992K3,1992Q3
3,1992K4,1992Q4
4,1993K1,1993Q1
...,...,...
119,2021K4,2021Q4
120,2022K1,2022Q1
121,2022K2,2022Q2
122,2022K3,2022Q3


We are only interested in a subset of the total dataset. Below, we specify the subset of the dataset we want to include

1. A param dictionary is defined in order to detaile the data we want

In [841]:
# Getting an overview of the underlying code that determines which variables and subset of data we import via the API
params = price._define_base_params(language='en')
params

{'table': 'ej55',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'EJENDOMSKATE', 'values': ['*']},
  {'code': 'TAL', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

1. We select the data we want. We only want data for "All Denmark" and indexed values, and percentage change compared to previous quarter.

In [842]:
# Using the format printed above to specify which subsets of the available dataset we want to import
params = {'table': 'ej55',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']},
  {'code': 'EJENDOMSKATE', 'values': ['0111']},
  {'code': 'TAL', 'values': ['100']}, # with the key 'code' we choose the desired variable and with the key 'values' we choose what subset of the dataset for the given variable we want to include
  {'code': 'Tid', 'values': ['*']}]} # ['*'] includes all the available data

1. Data is sorted and the index is reset. 
2. Columns are renamed.

In [843]:
sales_api = price.get_data(params=params) # retrieving the specified subset of the dataset from DST
sales_api.sort_values(by=['OMRÅDE', 'TID', 'EJENDOMSKATE'], inplace=True) # sorting the values
sales_api.reset_index(inplace = True, drop = True) # resetting the initial index, so it fits the new dataset
sales_api.rename(columns = {'OMRÅDE':'PROVINCE', 'EJENDOMSKATE':'CATEGORY', 'TAL':'UNIT', 'TID':'TIME', 'INDHOLD':'SALES_INDEX'}, inplace=True) # renaming columns

1. Missing values are replaced with NaN and then dropped.

In [844]:
sales_api = sales_api.replace('..', np.nan) # replacing all the missing data (denoted with '..' by DST) with NaN-values

1. Values types are replaced. 

In [845]:
sales_api.info() # assesing the types of data in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PROVINCE     1364 non-null   object
 1   CATEGORY     1364 non-null   object
 2   UNIT         1364 non-null   object
 3   TIME         1364 non-null   object
 4   SALES_INDEX  1308 non-null   object
dtypes: object(5)
memory usage: 53.4+ KB


1. The value variable is changed to a float type variable. 

In [846]:
sales_api.SALES_INDEX = sales_api.SALES_INDEX .astype('float') # changing the column SALES_INDEX from object to float
sales_api.info() # displaying the new types of variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PROVINCE     1364 non-null   object 
 1   CATEGORY     1364 non-null   object 
 2   UNIT         1364 non-null   object 
 3   TIME         1364 non-null   object 
 4   SALES_INDEX  1308 non-null   float64
dtypes: float64(1), object(4)
memory usage: 53.4+ KB


Below, we **explore the raw data** by creating **interactive plots** to show important developments 

**Interactive plot** :

1. We create an interactive plot of the housing prices in different provinces of Denmark

In [847]:
def plot_value(df, selected_provinces):
    fig, ax = plt.subplots()
    for province in selected_provinces:
        I = df['PROVINCE'] == province
        df.loc[I, :].plot(x='TIME', y='SALES_INDEX', legend=False, ax=ax)
    ax.set_xlabel('Time')
    ax.set_ylabel('Price Index')
    plt.legend(selected_provinces)
    plt.show()

options = sales_api.PROVINCE.unique()
province_checkboxes = {province: False for province in options}

widgets.interact(plot_value, 
                 df=widgets.fixed(sales_api),
                 selected_provinces=widgets.SelectMultiple(description='Provinces', options=options, value=[options[1]], disabled=False))


interactive(children=(SelectMultiple(description='Provinces', index=(1,), options=('Province Bornholm', 'Provi…

<function __main__.plot_value(df, selected_provinces)>

In the plot above, we see that all the (nominal) price indexes have followed an increasing trend since 1992. We note that the indexes are not comparable across regions, as each region is indexed so the value of the prices are normalized to index 100 in 2006. So, even if prices in e.g. Byen København are, say, 30 percent higher than prices in Fyn in 2006, both indexes will have the value of index 100.

However, from the graphical representation we can compare the relative increase in prices *within* a given province over time, and we see that the most dominant increases in prices since 2006 have been in the provinces 'Byen København', 'Københavns Omegn' and 'Bornholm'. In the recent quarters, of these three provinces, Bornholm have experienced the relatively largest decrease in prices again.

# Merge with data on gross unemployment

Now we wish to examine the correlation within provinces between gross unemployment and prices on one-family houses. Theory suggests that the gross unemployment in a province is a determinant of the housing prices, and that they are negatively correlated. The intuition is that when the unemployment rate increases in a province, then it becomes less attractive to move to this province as the risk of unemployment, all else equal, is higher in this area. Therefore, the demand of houses in the given province decreases implying that the housing prices decrease.

First, we collect data on gross unemployment within provinces in Denmark. This is done by using the API from Statistics Denmark and importing the register *AULK01*. 

Just like before, we then clean this dataset and select the relevant information from the dataset. To do this, we need to get an overview of the dataset:

In [848]:
#An overview over the available data. 
tabsum_unemp = unemp.tablesummary(language='en')
display(tabsum)

# The available values for each variable:
for variable in tabsum_unemp['variable name']:
    print(variable+':')
    display(unemp.variable_levels(variable, language='en'))

Table AUS08: unemployed persons (seasonally adjusted) by region, seasonal adjustment and actual figures and time
Last update: 2023-04-28T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,17,000,All Denmark,11,Province Nordjylland,False
1,EJENDOMSKATE,3,0111,One-family houses,2103,"Owner-occupied flats, total",False
2,TAL,3,100,Index,310,Percentage change compared to same quarter the...,False
3,Tid,124,1992K1,1992Q1,2022K4,2022Q4,True


OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,101,Copenhagen
4,147,Frederiksberg
...,...,...
112,787,Thisted
113,820,Vesthimmerlands
114,851,Aalborg
115,998,Unknown municipality


SAESONFAK:


,id,text
0,9,Seasonally adjusted figures in percent of the ...
1,10,Seasonally adjusted
2,22,Enumerated actual figures in percent of the l...
3,24,Enumerated actual figures


Tid:


,id,text
0,2007M01,2007M01
1,2007M02,2007M02
2,2007M03,2007M03
3,2007M04,2007M04
4,2007M05,2007M05
...,...,...
190,2022M11,2022M11
191,2022M12,2022M12
192,2023M01,2023M01
193,2023M02,2023M02


Now, we select the relevant variables for this analysis.

In [849]:
# Getting an overview of the underlying code that determines which variables and subset of data we import via the API
params_unemp = unemp._define_base_params(language='en')
params_unemp 

{'table': 'aus08',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'SAESONFAK', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [850]:
# Using the format printed above to specify which subsets of the available dataset we want to import
params_unemp = {'table': 'aus08',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']}, # with the key 'code' we choose the desired variable and with the key 'values' we choose what subset of the dataset for the given variable we want to include
  {'code': 'SAESONFAK', 'values': ['9']},
  {'code': 'Tid', 'values': ['*']}]} # ['*'] includes all the available values



In [851]:
unemp_api = unemp.get_data(params=params_unemp) # retrieving the specified subset of the dataset from DST
unemp_api.sort_values(by=['OMRÅDE', 'TID'], inplace=True) # sorting the values
unemp_api = unemp_api.drop(columns = ['SAESONFAK']) # dropping unwanted columns
unemp_api.reset_index(inplace = True, drop = True) # resetting the index, so it fits the new dataset
unemp_api.rename(columns = {'OMRÅDE':'PROVINCE','TID':'TIME', 'INDHOLD':'UNEMPLOYMENT_RATE'}, inplace=True) # renaming columns



### Taking yearly averages over the unemployment rate 

Now, we want to calculate yearly averages of the unemployment rate and the housing prices, respectively. This is so we can plot the two against each other later.

We start by creating a new variable 'YEAR' where it has the same value for all observations in a given year. Then we drop 'TIME' and change 'YEAR' to a float. Finally we calculate the average over each year. Firstly, we do this  the unemployment rate and then for the housing prices.

In [852]:
unemp_api['YEAR'] = pd.to_datetime(unemp_api['TIME'], format='%YM%m').dt.year # extracting the year information from the column 'TIME' and creating a new variable 'YEAR' with the year information
unemp_api['QUARTER'] = pd.to_datetime(unemp_api['TIME'], format='%YM%m').dt.quarter # extracting the quarter information from the column 'TIME' and creating a new variable 'YEAR' with the year information
#unemp_api = unemp_api.drop(columns = ['TIME']) # dropping the column 'TIME'
unemp_api.head(20) # displaying the first 10 rows of the dataset



,PROVINCE,TIME,UNEMPLOYMENT_RATE,YEAR,QUARTER
0,Province Bornholm,2007M01,7.8,2007,1
1,Province Bornholm,2007M02,7.8,2007,1
2,Province Bornholm,2007M03,7.7,2007,1
3,Province Bornholm,2007M04,7.7,2007,2
4,Province Bornholm,2007M05,7.8,2007,2
5,Province Bornholm,2007M06,7.6,2007,2
6,Province Bornholm,2007M07,7.3,2007,3
7,Province Bornholm,2007M08,7.2,2007,3
8,Province Bornholm,2007M09,7.1,2007,3
9,Province Bornholm,2007M10,7.1,2007,4


In [853]:
unemp_api['UNEMPLOYMENT_RATE'] = unemp_api['UNEMPLOYMENT_RATE'].astype('float') # changing the column UNEMPLOYMENT_RATE from object to float
unemp_api.info() # assesing the types of data in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PROVINCE           2145 non-null   object 
 1   TIME               2145 non-null   object 
 2   UNEMPLOYMENT_RATE  2145 non-null   float64
 3   YEAR               2145 non-null   int64  
 4   QUARTER            2145 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 83.9+ KB


In [854]:
unemp_avg = unemp_api.groupby(['PROVINCE', 'YEAR', 'QUARTER'])['UNEMPLOYMENT_RATE'].apply('mean') # calculating the average unemployment rate for each province in each year
unemp_avg = unemp_avg.reset_index() # resetting the index, so it fits the new dataset


unemp_avg['TIME'] = unemp_avg['YEAR'].astype(str) + 'Q' + unemp_avg['QUARTER'].astype(str) # creating a new variable 'TIME' with the year and quarter information
#dropping the columns 'YEAR' and 'QUARTER'
unemp_avg = unemp_avg.drop(columns = ['YEAR', 'QUARTER'])
unemp_avg.head(20)

,PROVINCE,UNEMPLOYMENT_RATE,TIME
0,Province Bornholm,7.766667,2007Q1
1,Province Bornholm,7.700000,2007Q2
2,Province Bornholm,7.200000,2007Q3
3,Province Bornholm,6.900000,2007Q4
4,Province Bornholm,6.600000,2008Q1
5,Province Bornholm,5.800000,2008Q2
6,Province Bornholm,5.500000,2008Q3
7,Province Bornholm,6.000000,2008Q4
8,Province Bornholm,6.900000,2009Q1
9,Province Bornholm,7.633333,2009Q2


Now, we also calculate the mean for the house prices in the same fashion.

Now, we wish to merge unemployment rate data within provinces on the dataset of housing prices. Here, we merge on province and year, such that we for each province in each year now also have information on the unemployment rate. 

Before we merge, we want to inspect in what years we have data in each of the two data sets. This is done below by printing the years which only exist in one of the data sets (we do this for each dataset) and then printing the provinces which only appear in one of the data sets (we do this for each dataset).



In [855]:
diff_y = [y for y in sales_api.TIME.unique() if y not in unemp_avg.TIME.unique()] #checking if there are any years in the sales_avg data that are not in the unemp_avg data
print(f'years in sales_avg data, but not in unemp_avg data: {diff_y}')

diff_y = [y for y in unemp_avg.TIME.unique() if y not in sales_api.TIME.unique()] #checking if there are any years in the unemp_avg data that are not in the sales_avg data
print(f'years in unemp_avg data, but not in sales_avg data: {diff_y}')

diff_m = [m for m in sales_api.PROVINCE.unique() if m not in unemp_avg.PROVINCE.unique()] #checking if there are any municipalities in the sales_avg data that are not in the unemp_avg data
print(f'provinces in sales_avg data, but not in unemp_avg data: {diff_m}')

diff_m = [m for m in unemp_avg.PROVINCE.unique() if m not in sales_api.PROVINCE.unique()] # Also checking if there are any municipalities in the unemp_avg data that are not in the sales_avg data
print(f'provinces in unemp_avg data, but not in sales_avg data: {diff_m}')

years in sales_avg data, but not in unemp_avg data: ['1992Q1', '1992Q2', '1992Q3', '1992Q4', '1993Q1', '1993Q2', '1993Q3', '1993Q4', '1994Q1', '1994Q2', '1994Q3', '1994Q4', '1995Q1', '1995Q2', '1995Q3', '1995Q4', '1996Q1', '1996Q2', '1996Q3', '1996Q4', '1997Q1', '1997Q2', '1997Q3', '1997Q4', '1998Q1', '1998Q2', '1998Q3', '1998Q4', '1999Q1', '1999Q2', '1999Q3', '1999Q4', '2000Q1', '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2', '2001Q3', '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4', '2003Q1', '2003Q2', '2003Q3', '2003Q4', '2004Q1', '2004Q2', '2004Q3', '2004Q4', '2005Q1', '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2', '2006Q3', '2006Q4']
years in unemp_avg data, but not in sales_avg data: ['2023Q1']
provinces in sales_avg data, but not in unemp_avg data: []
provinces in unemp_avg data, but not in sales_avg data: []


We see that we have data on housing prices from 1992 and onwards, while we only have data on unemployment rate from 2007. Likewise, we only have a yearly average on the unemployment rate for 2023 (which is only a three-month average). 

As all observations on housing prices before 2007Q1 cannot be linked to the unemployment rate in the given period and vice versa for the unemployment rate on housing prices in 2023, these observations are irrelevant. We therefore use the `inner join`-method to merge. This method only includes the matches between the two datasets. 

In [856]:
sales_with_unemp = pd.merge(sales_api, unemp_avg, on = ['PROVINCE', 'TIME'], how = 'inner') # Performing an inner merge of the two datasets (keeping only data for which there are observations for both variables in the dataset)
sales_with_unemp.sample(10) # displaying a sample of 10 random values in the dataset

,PROVINCE,CATEGORY,UNIT,TIME,SALES_INDEX,UNEMPLOYMENT_RATE
457,Province Vest- og Sydsjælland,One-family houses,Index,2009Q2,88.9,5.033333
81,Province Byen København,One-family houses,Index,2011Q2,84.6,7.133333
380,Province Nordsjælland,One-family houses,Index,2022Q1,126.9,1.933333
79,Province Byen København,One-family houses,Index,2010Q4,81.5,7.366667
339,Province Nordsjælland,One-family houses,Index,2011Q4,70.4,4.400000
415,Province Sydjylland,One-family houses,Index,2014Q4,95.1,4.333333
384,Province Sydjylland,One-family houses,Index,2007Q1,106.8,3.133333
59,Province Bornholm,One-family houses,Index,2021Q4,150.4,2.433333
352,Province Nordsjælland,One-family houses,Index,2015Q1,79.3,3.600000
142,Province Fyn,One-family houses,Index,2010Q3,100.8,7.033333


Now that the dataset is ready, we can examine the correlation between gross unemployment and prices on one-family houses within each province. We do this by constructing a binned scatterplot for each province. The binned scatterplot groups the unemployment rates into 10 equal-sized bins for each province and plots the mean of the associated housing prices within each bin. 

We construct the binned scatter plot using the package *binsreg*, which can be installed writing "pip install binsreg" in the prompt.

In [857]:
# Import binsreg
import binsreg

def binscatter(df,province):
    I =df[df['PROVINCE'] == province]
    binsreg.binsreg('SALES_INDEX', 'GROSS UNEMPLOYMENT', data=I, 
                    nbins=10, #specify 10 bins 
                    polyreg=1, #create linear fitted line      
    )

    
widgets.interact(binscatter, # creating interactive widget letting us choose the desired province
    df = widgets.fixed(sales_with_unemp),
    province = widgets.Dropdown(description='Province', 
                                    options=sales_with_unemp.PROVINCE.unique(), 
                                    value='Province Byen København'),
)



interactive(children=(Dropdown(description='Province', index=1, options=('Province Bornholm', 'Province Byen K…

<function __main__.binscatter(df, province)>

From the interactive figure above, we clearly observe a negative correlation between gross unemployment rate and housing prices in all of the provinces. This results supports the theoretical suggestions that the gross unemployment rate in a province is a determinant of the housing prices in the province. However, it is important to emphasize that these results are only correlations, hence we cannot give the results any causal interpretations. In principle, the negative correlation could be due to reverse causality or omitted variables correlating with both gross unemployment and the housing prices (e.g. the interest rates).